In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df_log = pd.read_csv("CGS_data/data/SRP073813/SRP073813_log.csv")
df_meta = pd.read_csv("CGS_data/data/SRP073813/SRP073813_meta.csv")

In [3]:
df_log.set_index("HUGO", inplace = True)
df_meta.set_index("ID", inplace = True) #Setting the index for data makes the enrichment analysis easier
df_meta = df_meta['refinebio_subject'] #removes unnecessary data 

In [4]:
df_meta = df_meta.to_frame()

In [5]:
df_log =round(df_log) 
df_log = df_log.T #swaps columns and indexes
df_log = df_log.join(df_meta)

In [6]:
df_log = df_log.loc[:, ~df_log.columns.duplicated()] #removes duplicate columns prevent deseq

In [7]:
#Transfomring the data to contain only data collected from a certain part of the brain in our patients 
df_log = df_log[df_log["refinebio_subject"].str.contains('nacc')]
df_meta = df_meta[df_meta["refinebio_subject"].str.contains('nacc')] 

In [8]:
df_log.drop("refinebio_subject",axis =1,inplace = True)

In [9]:
df_log = df_log.T

In [10]:
#get variacne sort by it and select the top ones to do Hclust on
df_log["variance"] = np.var(df_log.values, axis=1)
df = df_log.sort_values("variance", ascending=False)
df.drop("variance", axis =1, inplace = True)
dfTen = df.head(10).T
dfHun = df.head(100).T
dfThous = df.head(1000).T
dfTenThous = df.head(10000).T
dfFiveThous = df.head(5000).T


In [11]:
dfFiveThous = dfFiveThous.join(df_meta)
dfTen = dfTen.join(df_meta)
dfHun = dfHun.join(df_meta)
dfThous = dfThous.join(df_meta)
dfTenThous = dfTenThous.join(df_meta)

In [12]:
def create_model(df):
    y = df.refinebio_subject
    cols = np.delete(df.columns, len(df.columns)-1,0)
    X = df[cols]
    forest_model = RandomForestClassifier(random_state=1)
    train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.5, random_state=1)
    forest_model.fit(train_X,train_y)
    y_pred = forest_model.predict(val_X)
    accuracy = accuracy_score(val_y, y_pred)
    print(f"{len(df.columns)-1} Genes Accuracy: {accuracy:.2f}")
    forest_model.fit(X,y)
    return forest_model
    

In [13]:
tenModel =create_model(dfTen)
hunModel = create_model(dfHun)
thousModel=create_model(dfThous)
fiveThousModel=create_model(dfFiveThous)
tenThousModel = create_model(dfTenThous)

10 Genes Accuracy: 0.40
100 Genes Accuracy: 0.39
1000 Genes Accuracy: 0.40
5000 Genes Accuracy: 0.49
10000 Genes Accuracy: 0.35


In [14]:
cols = np.delete(dfFiveThous.columns, len(dfFiveThous.columns)-1,0)
X = dfFiveThous[cols]

In [15]:
def extract_genes(df,model, threshold):
    gene_names = np.delete(df.columns, len(df.columns)-1,0)
    importances = model.feature_importances_
    significant_genes_rf = [gene_names[i] for i, importance in enumerate(importances) if importance > threshold]
    return significant_genes_rf

In [16]:
sig_10 = set(extract_genes(dfTen, tenModel, 0.00125))
sig_100 = set(extract_genes(dfHun, hunModel, 0.00125))
sig_1000 = set(extract_genes(dfThous, thousModel, 0.00125))
sig_5000 =set(extract_genes(dfFiveThous, fiveThousModel, 0.00125))
sig_10000 = set(extract_genes(dfTenThous,tenThousModel, 0.00125))

In [17]:
import venn

# Define your sets
sets = {
    '10': set(sig_10),
    '100': set(sig_100),
    '1000': set(sig_1000),
    '5000': set(sig_5000),
    '10000': set(sig_10000)
}
for val in sets:
    print(f"There are {len(sets[val])} significant genes when look at the {val} most variable genes")


There are 10 significant genes when look at the 10 most variable genes
There are 98 significant genes when look at the 100 most variable genes
There are 311 significant genes when look at the 1000 most variable genes
There are 98 significant genes when look at the 5000 most variable genes
There are 57 significant genes when look at the 10000 most variable genes


In [18]:
impGenes = dfFiveThous[list(sig_5000)]
impGenes = impGenes.join(df_meta)

In [19]:
impGenes.to_csv("CGS_data/data/SRP073813/model_genes.csv")

In [20]:
print(sig_10000.intersection(sig_1000).intersection(sig_100).intersection(sig_10))

set()


In [21]:
#Logictic Regresion
def make_model(df):
    y = df.refinebio_subject
    cols = np.delete(df.columns, len(df.columns)-1,0)
    x = df[cols]
    model = LogisticRegression(solver='newton-cg', multi_class='multinomial', random_state=0)
    x_train, x_test, y_train, y_test =\
    train_test_split(x, y, test_size=0.2, random_state=1)
    model.fit(x_train, y_train) 
    y_pred = model.predict(x_test)
    accuracy = model.score(x_test, y_test)
    print(f"{len(df.columns)-1} Genes Accuracy: {accuracy:.2f}")
    model.fit(x,y)
    return model

In [22]:
log5000m = make_model(dfFiveThous)

5000 Genes Accuracy: 0.43


In [23]:
def log_extract_genes(df, model, threshold):
    gene_names = np.delete(df.columns, len(df.columns)-1,0)
    importances = model.coef_[0]
    significant_genes_rf = [gene_names[i] for i, coef in enumerate(importances) if abs(coef) > threshold]
    return significant_genes_rf

In [24]:
log_sig_5000 = set(log_extract_genes(dfFiveThous,log5000m, 0.0015))

In [37]:
len(log_sig_5000)

4578

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def data_split(data1, data2):
    d3 = pd.concat([data1,data2], axis=0).sample(frac=1.0, random_state=42)
    return train_test_split(d3.values,d3.index.values, test_size=0.25,random_state=42)

def data_group(data):
    return data.loc['nacc_schizophrenia'], data.loc['nacc_major depression'], data.loc['nacc_bipolar disorder'], data.loc['nacc_control']

def model_predict(data):
    xTrain,xTest,yTrain, yTest = data
    svm_classifier = SVC(kernel='linear')
    svm_classifier.fit(xTrain, yTrain)
    y_pred = svm_classifier.predict(xTest)
    return accuracy_score(yTest, y_pred)

def test_for_accuracy(data,numGenes):
    dS, dD, dB, dC = data_group(data)
    print("Model accuracy for top " + numGenes + " genes, classifying Schizophrenia vs. Control: " + str(model_predict(data_split(dS,dC))))
    print("Model accuracy for top " + numGenes + " genes, classifying Major Depression vs. Control: " + str(model_predict(data_split(dD,dC))))
    print("Model accuracy for top " + numGenes + " genes, classifying Bipolar Disorder vs. Control: " + str(model_predict(data_split(dB,dC))))
    print()
    

In [27]:
from sklearn.feature_selection import RFECV
def data_train(data1, data2):
    d3 = pd.concat([data1,data2], axis=0).sample(frac=1.0, random_state=42)
    return d3.values, d3.index.values

def computeSigFeatures(data, step_val, cv_val):
    X_train, y_train = data
    svm = SVC(kernel='linear')
    rfecv = RFECV(estimator=svm, step=step_val, cv=cv_val, scoring='accuracy')
    rfecv.fit(X_train, y_train)
    selected_gene_indices = rfecv.support_
    return selected_gene_indices.tolist()
    
def modelFeatures(data, numGenes, step, cv):
    dS, dD, dB, dC = data_group(data)
    gene_subset = data.columns.tolist()
    featuresS = computeSigFeatures(data_train(dS,dC), step, cv)
    featuresD = computeSigFeatures(data_train(dD,dC), step, cv)
    featuresB = computeSigFeatures(data_train(dB,dC), step, cv)
    sigS = [item for item, should_keep in zip(gene_subset, featuresS) if should_keep]
    sigD = [item for item, should_keep in zip(gene_subset, featuresD) if should_keep]
    sigB = [item for item, should_keep in zip(gene_subset, featuresB) if should_keep]
    print("Number of significant features for top " + numGenes + " variable genes, for classifying Schizophrenia vs. Control: " + str(len(sigS)))
    print("Number of significant features for top " + numGenes + " variable genes, for classifying Major Depression vs. Control: " + str(len(sigD)))
    print("Number of significant features for top " + numGenes + " variable genes, for classifying Bipolar Disorder vs. Control: " + str(len(sigB)))
    print()
    return sigS, sigD, sigB
    

In [28]:
temp = dfFiveThous
dfFiveThous = dfFiveThous.set_index("refinebio_subject")
mf5000 = modelFeatures(dfFiveThous, "5000", 10, 3)


Number of significant features for top 5000 variable genes, for classifying Schizophrenia vs. Control: 90
Number of significant features for top 5000 variable genes, for classifying Major Depression vs. Control: 1530
Number of significant features for top 5000 variable genes, for classifying Bipolar Disorder vs. Control: 1



In [36]:
sv_5000 = set()
for arr in mf5000:
    for val in arr:
        sv_5000.add(val)
len(sv_5000)

1567

In [30]:
important_genes =log_sig_5000.intersection(sig_5000).intersection(sv_5000)
len(important_genes)

49

In [31]:
impGenes = temp[list(important_genes)]
impGenes = impGenes.join(df_meta)

In [35]:
impGenes

,ACBD4,UBE2F-SCLY,TRIO,FLT1,DPP7,OBSCN,RN7SL455P,MFRP,SIX1,EDNRB,...,F3,TNRC18,SEMA3B,POLE3,ARPP19,RTL6,PPIAL4E,SUN2,TNPO2,refinebio_subject
SRR3438559,2.0,1.0,4.0,4.0,4.0,5.0,2.0,1.0,2.0,3.0,...,2.0,5.0,5.0,3.0,5.0,4.0,2.0,5.0,6.0,nacc_schizophrenia
SRR3438560,2.0,1.0,3.0,3.0,4.0,5.0,1.0,1.0,1.0,3.0,...,3.0,5.0,4.0,2.0,6.0,4.0,2.0,6.0,6.0,nacc_control
SRR3438561,2.0,2.0,4.0,3.0,4.0,5.0,1.0,1.0,1.0,3.0,...,3.0,5.0,4.0,2.0,5.0,3.0,1.0,5.0,6.0,nacc_control
SRR3438562,3.0,2.0,4.0,3.0,4.0,5.0,2.0,2.0,2.0,3.0,...,3.0,5.0,3.0,3.0,6.0,4.0,2.0,4.0,5.0,nacc_schizophrenia
SRR3438615,3.0,2.0,3.0,3.0,4.0,5.0,1.0,1.0,2.0,4.0,...,3.0,5.0,4.0,3.0,6.0,3.0,1.0,5.0,6.0,nacc_control
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR3438899,2.0,1.0,4.0,3.0,4.0,5.0,2.0,1.0,2.0,3.0,...,3.0,5.0,3.0,3.0,6.0,3.0,1.0,4.0,6.0,nacc_major depression
SRR3438902,2.0,1.0,4.0,4.0,4.0,3.0,2.0,1.0,1.0,3.0,...,3.0,5.0,6.0,2.0,4.0,3.0,2.0,6.0,5.0,nacc_schizophrenia
SRR3438903,2.0,1.0,4.0,4.0,4.0,6.0,2.0,2.0,2.0,5.0,...,3.0,5.0,4.0,2.0,5.0,3.0,2.0,5.0,6.0,nacc_schizophrenia
SRR3438904,2.0,1.0,4.0,4.0,4.0,5.0,2.0,1.0,2.0,2.0,...,2.0,6.0,4.0,2.0,3.0,2.0,2.0,4.0,6.0,nacc_schizophrenia


In [34]:
impGenes.to_csv("CGS_data/data/SRP073813/model_genes.csv")